# 1. Import libraries and load the dataset



In [ ]:
import pandas as pd
import numpy as np


df_holidays_events = pd.read_csv(r'D:\Topic_13_Project\Topic_13_Retail_Store_Sales_Time_Series\data\raw\holidays_events.csv')

# 2. Display basic information about the dataset


In [6]:
print("First 5 Rows of Data Frame:\n", df_holidays_events.head(5))
print("Data Frame Shape:\n", df_holidays_events.shape)
print("Data Frame Info:\n", df_holidays_events.info())
print("Data Frame Statistics:\n", df_holidays_events.describe())
    

First 5 Rows of Data Frame:
          date     type    locale locale_name                    description  \
0  2012-03-02  Holiday     Local       Manta             Fundacion de Manta   
1  2012-04-01  Holiday  Regional    Cotopaxi  Provincializacion de Cotopaxi   
2  2012-04-12  Holiday     Local      Cuenca            Fundacion de Cuenca   
3  2012-04-14  Holiday     Local    Libertad      Cantonizacion de Libertad   
4  2012-04-21  Holiday     Local    Riobamba      Cantonizacion de Riobamba   

   transferred  
0        False  
1        False  
2        False  
3        False  
4        False  
Data Frame Shape:
 (350, 6)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         350 non-null    object
 1   type         350 non-null    object
 2   locale       350 non-null    object
 3   locale_name  350 non-null    object
 4   description

# 3. Missing Values Summary

In [7]:
# Calculate missing values
missing_counts = df_holidays_events.isnull().sum()
missing_percentage = missing_counts / len(df_holidays_events) * 100

# Create a summary DataFrame
missing_df_holidays_events = pd.DataFrame({
    'Missing Values': missing_counts,
    'Percentage': missing_percentage
})

print("Missing Values Summary:\n", missing_df_holidays_events)

# Filter columns with missing values
missing_df_holidays_events = missing_df_holidays_events[missing_df_holidays_events['Missing Values'] > 0].sort_values(by='Missing Values', ascending=False)

Missing Values Summary:
              Missing Values  Percentage
date                      0         0.0
type                      0         0.0
locale                    0         0.0
locale_name               0         0.0
description               0         0.0
transferred               0         0.0
